## Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

Many thanks to [`mateusvmv`](https://github.com/mateusvmv) for this hack in [`gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3`](https://gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3?permalink_comment_id=4715636#gistcomment-4715636).

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Install Dependencies

In [2]:
:dep ndarray = {version = "0.15.6"}
// or
// :dep ndarray = { git = "https://github.com/rust-ndarray/ndarray"}

In [6]:
:dep blas-src = { version = "0.9", features = ["openblas"]}
// or
// :dep blas-src = { git = "https://github.com/blas-lapack-rs/blas-src", features = ["openblas"]}

In [7]:
:dep ndarray-linalg = { version = "0.16.0", features = ["openblas-static"]}
// or
// :dep ndarray-linalg = { git = "https://github.com/rust-ndarray/ndarray-linalg", features = ["openblas-static"]}

In [8]:
:dep ndarray-rand = {version = "0.14.0"}
// or
// :dep ndarray-rand = { git = "https://github.com/rust-ndarray/ndarray/tree/ndarray-rand-0.14.0"}

## Import Libraries

In [9]:
use std::collections::{HashSet, HashMap};
use ndarray::{array, Array, Array1, Array2, Array3, ShapeBuilder, rcarr1};
use ndarray_linalg::convert::flatten;
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use ndarray_linalg::solve::Inverse;
use ndarray_linalg::solve::Determinant;
use ndarray_linalg::trace::Trace;
use ndarray_linalg::Solve;
use ndarray_linalg::svd::SVD;
use ndarray_linalg::Eig;
use std::result::Result::{Err, Ok};

<hr />

# 1. Vectors

## Creating a Vector

In [10]:
let v: Vec<i32> = Vec::new();
v

[]

In [11]:
let v = vec![1, 2, 3];
v

[1, 2, 3]

## Accessing Vectors Elements

In [12]:
let v = vec!["apple", "banana", "cherry", "date"];
{
    let second = &v[1];
    println!("{second}")   
}

banana


()

In [13]:
let v = vec![
    ("apple", 3),
    ("banana", 2),
    ("cherry", 5),
    ("date", 1),
];

{
    // Get the quantity of cherries
    let quantity = v.get(2).map(|(_, q)| q);

    match quantity {
        Some(q) => println!("There are {} cherries", q),
        None => println!("Cherries not found"),
    }
}

There are 5 cherries


()

## Iterating over Values

In [14]:
let fruits = vec![("apple", 3), ("banana", 2), ("orange", 5), ("peach", 4)];
let mut sum = 0;
for (_, num) in &fruits {
    sum += num;
}
let avg = sum as f32 / fruits.len() as f32;
avg

3.5

In [15]:
let mut values = vec![10, 20, 30, 40, 50];
for value in values.iter_mut() {
    *value += 10;
}
values

[20, 30, 40, 50, 60]

In [16]:
let values = vec![10, 20, 30, 40, 50];
for value in &values[0..3] {
    println!("The value is {}", value);
}

The value is 10
The value is 20
The value is 30


()

In [17]:
let values = vec![10, 20, 30, 40, 50];
for (index, value) in values.iter().enumerate() {
    println!("The value at index {} is {}", index, value);
}

The value at index 0 is 10
The value at index 1 is 20
The value at index 2 is 30
The value at index 3 is 40
The value at index 4 is 50


()

## Modifying a Vector

## Adding elements

In [18]:
let mut v = vec!["apple", "banana", "orange"];
v.push("mango");
v

["apple", "banana", "orange", "mango"]

In [19]:
let mut v = vec!["apple", "mango", "banana", "orange"];
v.insert(v.len(), "mango");
v

["apple", "mango", "banana", "orange", "mango"]

## Modifying Elements

In [20]:
let mut v = vec!["apple", "banana", "orange"];
v[1] = "pear";
v[2] = "grapefruit";
v

["apple", "pear", "grapefruit"]

## Removing Elements

In [21]:
let mut v = vec!["apple", "banana", "orange", "mango"];
let removed_element = v.pop();
println!("Removed element: {:?}", removed_element.unwrap());
println!("{:?}", v);

Removed element: "mango"
["apple", "banana", "orange"]


In [22]:
let mut v = vec!["apple", "banana", "orange", "mango"];
let removed_element = v.remove(2);
println!("Removed element: {}", removed_element);
println!("{:?}", v);

Removed element: orange
["apple", "banana", "mango"]


In [23]:
let mut v = vec!["A", "warm", "fall", "warm", "day"];
let elem = "warm"; // element to remove
v.retain(|x| *x != elem);
println!("{:?}", v);

["A", "fall", "day"]


In [24]:
let mut v1 = vec!["apple", "banana"];
let mut v2 = vec!["orange", "mango"];
v1.extend(v2);
println!("{:?}", v1);

["apple", "banana", "orange", "mango"]


## Filter & Map Elements

In [25]:
let v = vec![1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
let odd_numbers: Vec<i32> = v.iter().filter(|x| *x % 2 != 0).map(|x| *x).collect();
println!("{:?}", odd_numbers);

[1, 3, 5, 7, 9]


In [26]:
let v = vec!["hello", "world", "rust"];
let uppercase_strings: Vec<String> = v.iter().map(|x| x.to_uppercase()).collect();
println!("{:?}", uppercase_strings);

["HELLO", "WORLD", "RUST"]


## Vector Length

In [27]:
let v = vec!["hello", "world", "rust"];
println!("Size: {}", v.len());

Size: 3


## Check If Element Exists

In [28]:
let v = vec!["hello", "world", "rust"];
println!("{}", v.contains(&"hello"));

true


## Reversing Elements

In [29]:
let mut v = vec![1, 2, 3, 4, 5];
v.reverse();
println!("{:?}", v);

[5, 4, 3, 2, 1]


## Maximum & Minimum Elements

In [30]:
let v = vec![1, 2, 3, 4, 5];
let max_element = *v.iter().max().unwrap();
let min_element = *v.iter().min().unwrap();
println!("Max element: {}", max_element);
println!("Min element: {}", min_element);

Max element: 5
Min element: 1


<hr />

# 2. Arrays

## Creating an array

In [31]:
let days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"];
days

["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

In [32]:
let a: [i32; 5] = [1, 2, 3, 4, 5];
a

[1, 2, 3, 4, 5]

In [33]:
let zeros = [0; 5];
zeros

[0, 0, 0, 0, 0]

## Accessing Elements

In [34]:
let numbers = [1, 2, 3, 4, 5];
println!("{}", numbers[2]);

3


## Modifying Elements

In [35]:
let mut numbers = [1, 2, 3, 4, 5];
numbers[1] = 10;
println!("{:?}", numbers);

[1, 10, 3, 4, 5]


## Iterating

In [36]:
let seasons = ["Winter", "Spring", "Summer", "Fall"];
for season in seasons {
    println!("{season}");
}
// or
for index in 0..seasons.len() {
    println!("{}", seasons[index]);
}
// or
for season in seasons.iter() {
    println!("{}", season);
}

Winter
Spring
Summer
Fall
Winter
Spring
Summer
Fall
Winter
Spring
Summer
Fall


()

## Slicing Arrays

In [37]:
let numbers = [1, 2, 3, 4, 5];
{
    let slice = &numbers[1..4];
    println!("{:?}", slice);   
}

[2, 3, 4]


()

<hr />

# 3. Tuples

## Creating a Tuple

In [38]:
let person = ("Mahmoud", 22, true, 6.6);
person

("Mahmoud", 22, true, 6.6)

In [39]:
person

("Mahmoud", 22, true, 6.6)

## Updating a Tuple

In [40]:
let mut person = ("Mahmoud", 22, true);
person

("Mahmoud", 22, true)

In [41]:
person.1 = 21;
person

("Mahmoud", 21, true)

## Destructuring a Tuple

In [42]:
let (name, age, is_male) = ("Mahmoud", 22, true);
println!("Name: {}, Age: {}, Gender: {}", name, age, if is_male { "Male" } else { "Female" });

Name: Mahmoud, Age: 22, Gender: Male


In [43]:
let (_, _, _, height) = ("Mahmoud", 22, false, 6.6);
println!("Height: {}", height);

Height: 6.6


In [44]:
let person = ("Mahmoud", 3, true, 6.0);
println!("Experience: {}", person.1);

Experience: 3


<hr />

# 4. Hash Sets

## Creating a Set

In [45]:
let mut my_set: HashSet<i32> = HashSet::new();
my_set

{}

In [46]:
let my_vector = vec![1, 2, 3, 4];
let my_set: HashSet<i32> = my_vector.into_iter().collect();
my_set

{4, 1, 2, 3}

In [47]:
let a = HashSet::from([1, 2, 3]);
a

{1, 2, 3}

In [48]:
let mut my_set: HashSet<i32> = HashSet::new();
my_set.insert(1);
my_set.insert(2);
my_set.insert(3);
my_set

{1, 2, 3}

## Removing elements

In [49]:
let mut my_set = HashSet::from([1, 2, 3, 4]);
my_set.remove(&2); // removes 2 from the set
my_set

{1, 4, 3}

## Iterate over Sets

In [50]:
let my_set = HashSet::from([1, 2, 3]);

for element in &my_set {
    println!("{}", element);
}

3
1
2


()

## Sets Operations

In [51]:
let set_a = HashSet::from([1, 2, 3]);
let set_b = HashSet::from([4, 2, 3, 4]);

// elements in set_a that are not in set_b
let difference_set = set_a.difference(&set_b);

// elements common to both set_a and set_b
let intersection = set_a.intersection(&set_b);

// elements in either set_a or set_b
let union_set = set_a.union(&set_b);

println!("Difference of both sets: ");
    
for element in difference_set {
    println!("{}", element);
}

println!("Intersection of both sets: ");

for element in intersection {
    println!("{}", element);
}

println!("Union of both sets: ");

for element in union_set {
    println!("{}", element); 
}

Difference of both sets: 
1
Intersection of both sets: 
3
2
Union of both sets: 
3
1
2
4


()

# 5. HashMaps

## Creating a Hash Map

In [52]:
let mut employees_map = HashMap::new();

// Insert elements to the HashMap
employees_map.insert("Mahmoud", 1);
employees_map.insert("Ferris", 2);

employees_map

{"Mahmoud": 1, "Ferris": 2}

In [53]:
let employees_map: HashMap<i32, &str> = HashMap::from([
    (1, "Mahmoud"),
    (2, "Ferris"),
]);
employees_map

{1: "Mahmoud", 2: "Ferris"}

## Updating a Hash Map

## Adding Elements

In [54]:
let mut employees_map = HashMap::new();
// Insert elements to the HashMap
employees_map.insert("Mahmoud", 1);
employees_map.insert("Ferris", 2);
employees_map

{"Mahmoud": 1, "Ferris": 2}

## Removing Elements

In [55]:
let mut employees_map: HashMap<i32, String> = HashMap::new();

// insert elements to hashmap
employees_map.insert(1, String::from("Mahmoud"));

// remove elements from hashmap
employees_map.remove(&1);
employees_map

{}

## Updating an Element

In [56]:
let mut employees_map: HashMap<i32, String> = HashMap::new();

// insert elements to hashmap
employees_map.insert(1, String::from("Mahmoud"));

// update the value of the element with key 1
employees_map.insert(1, String::from("Ferris"));
employees_map

{1: "Ferris"}

## Access Values

In [57]:
let employees_map: HashMap<i32, &str> = HashMap::from([
    (1, "Mahmoud"),
    (2, "Ferris"),
]);

{
    let first_employee = employees_map.get(&1);
    first_employee.unwrap()
}

"Mahmoud"

## Iterate over Hash Maps

In [58]:
let mut employees_map: HashMap<i32, String> = HashMap::new();

employees_map.insert(1, String::from("Mahmoud"));
employees_map.insert(2, String::from("Ferris"));
    
// loop and print values of hashmap using values() method
for employee in employees_map.values() {
    println!("{}", employee)
}
    
// print the length of hashmap using len() method
println!("Length of employees_map = {}", employees_map.len());

Ferris
Mahmoud
Length of employees_map = 2


<hr />

# Ndarray for Data Analysis

## Initial Placeholders

### Zeros

In [59]:
let zeros = Array::<f64, _>::zeros((1, 4).f());
zeros

[[0.0, 0.0, 0.0, 0.0]], shape=[1, 4], strides=[1, 1], layout=CFcf (0xf), const ndim=2

### Ones

In [60]:
let ones = Array::<f64, _>::ones((1, 4));
ones

[[1.0, 1.0, 1.0, 1.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

### Range

In [61]:
let range = Array::<f64, _>::range(0., 5., 1.);
range

[0.0, 1.0, 2.0, 3.0, 4.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1

### Linspace

In [62]:
let linspace = Array::<f64, _>::linspace(0., 5., 5);
linspace

[0.0, 1.25, 2.5, 3.75, 5.0], shape=[5], strides=[1], layout=CFcf (0xf), const ndim=1

### Fill

In [63]:
let mut ones = Array::<f64, _>::ones((1, 4));
ones.fill(2.);
ones

[[2.0, 2.0, 2.0, 2.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

### Eye

In [64]:
let eye = Array::<f64, _>::eye(4);
eye

[[1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0]], shape=[4, 4], strides=[4, 1], layout=Cc (0x5), const ndim=2

### Random

In [65]:
let random = Array::random((2, 5), Uniform::new(0., 10.));
random

[[3.8224106202506203, 6.64094720699246, 9.454368791520007, 2.3125543314463393, 1.396546223588786],
 [1.648374869902045, 8.209199239430019, 3.486911941465447, 4.402403117231064, 5.900647270948909]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

<hr />

## Multidimensional Arrays

### 1D array

In [66]:
let array_d1 = Array::from_vec(vec![1., 2., 3., 4.]);
array_d1

[1.0, 2.0, 3.0, 4.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1

In [67]:
let array_d11 = Array::from_shape_vec((1, 4), vec![1., 2., 3., 4.]);
array_d11.unwrap()

[[1.0, 2.0, 3.0, 4.0]], shape=[1, 4], strides=[4, 1], layout=CFcf (0xf), const ndim=2

In [68]:
let array_d12 = array![
    [-1.01,  0.86, -4.60,  3.31, -4.81]
];
array_d12

[[-1.01, 0.86, -4.6, 3.31, -4.81]], shape=[1, 5], strides=[5, 1], layout=CFcf (0xf), const ndim=2

### 2D array

In [69]:
let array_d2 = array![
    [-1.01,  0.86, -4.60,  3.31, -4.81],
    [ 3.98,  0.53, -7.04,  5.29,  3.55],
    [ 3.30,  8.26, -3.89,  8.20, -1.51],
    [ 4.43,  4.96, -7.66, -7.33,  6.18],
    [ 7.31, -6.43, -6.16,  2.47,  5.58],
];
array_d2

[[-1.01, 0.86, -4.6, 3.31, -4.81],
 [3.98, 0.53, -7.04, 5.29, 3.55],
 [3.3, 8.26, -3.89, 8.2, -1.51],
 [4.43, 4.96, -7.66, -7.33, 6.18],
 [7.31, -6.43, -6.16, 2.47, 5.58]], shape=[5, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

In [70]:
let array_d21 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
array_d21.unwrap()

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

In [71]:
let mut data = vec![1., 2., 3., 4.];
let array_d22 = Array2::from_shape_vec((2, 2), data);
array_d22.unwrap()

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

### 3D array

In [72]:
let mut data = vec![1., 2., 3., 4.];
let array_d3 = Array3::from_shape_vec((2, 2, 1), data);
array_d3.unwrap()

[[[1.0],
  [2.0]],

 [[3.0],
  [4.0]]], shape=[2, 2, 1], strides=[2, 1, 1], layout=Cc (0x5), const ndim=3

<hr />

## Ndarray Arrays Manipulation

### Indexing

In [73]:
let array_d1 = Array::from_vec(vec![1., 2., 3., 4.]);
array_d1[1]

2.0

In [74]:
let zeros = Array2::<f64>::zeros((2, 4));
zeros[[1, 1]]

0.0

<hr />

## Reshaping

In [75]:
let array_d1 = rcarr1(&[1., 2., 3., 4.]); // another way to create a 1D array
let array_d2 = array_d1.reshape((2, 2));
array_d2

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

## Flatten

In [76]:
let array_d2: Array2<f64> = array![[3., 2.], [2., -2.]];
let array_flatten = flatten(array_d2);
array_flatten

[3.0, 2.0, 2.0, -2.0], shape=[4], strides=[1], layout=CFcf (0xf), const ndim=1

## Transposing

In [77]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
array_d2.unwrap()

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

In [78]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
let binding = array_d2.expect("Expect 2d matrix");
{
    let array_d2t = binding.t();
    array_d2t
}

[[1.0, 3.0],
 [2.0, 4.0]], shape=[2, 2], strides=[1, 2], layout=Ff (0xa), const ndim=2

## Swapping Axes

In [79]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
array_d2.unwrap()

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

In [80]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 3., 4.]);
let mut binding = array_d2.expect("Expect 2d matrix");
binding.swap_axes(0, 1);
binding

[[1.0, 3.0],
 [2.0, 4.0]], shape=[2, 2], strides=[1, 2], layout=Ff (0xa), const ndim=2

<hr />

## Linear Algebra

### 1. Matrix Multiplication

In [81]:
extern crate blas_src;

let a: Array2<f64> = array![[3., 2.], [2., -2.]];
let b: Array2<f64> = array![[3., 2.], [2., -2.]];
let c = a.dot(&b);
c

[[13.0, 2.0],
 [2.0, 8.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

### 2. Inversion

In [82]:
let array_d2 = Array::from_shape_vec((2, 2), vec![1., 2., 2., 1.]);

match array_d2.expect("Matrix must be square & symetric!").inv() {
    Ok(inv) => {
        println!("The inverse of m1 is: {}", inv);
    }
    Err(err) => {
        println!("{err}");
    }
}

The inverse of m1 is: [[-0.3333333333333333, 0.6666666666666666],
 [0.6666666666666666, -0.3333333333333333]]


()

### 3. Eigen Decomposition

In [83]:
let array_d2 = array![
    [-1.01,  0.86, -4.60],
    [ 3.98,  0.53, -7.04],
    [ 3.98,  0.53, -7.04],
];
match array_d2.eig() {
    Ok((eigs, vecs)) => {
        println!("Eigen values: {}", eigs);
        println!("Eigen vectors: {}", vecs);
    }
    Err(err) => {
        println!("{err}");
    }
}

Eigen values: [-3.759999999999999+2.706048780048134i, -3.759999999999999-2.706048780048134i, 0.00000000000000022759891370571733+0i]
Eigen vectors: [[0.402993672209733+0.3965529218364603i, 0.402993672209733-0.3965529218364603i, 0.13921180485702092+0i],
 [0.5832417510526318+0.00000000000000006939572631647882i, 0.5832417510526318-0.00000000000000006939572631647882i, 0.9784706726517249+0i],
 [0.583241751052632+-0i, 0.583241751052632+0i, 0.15236540338584623+0i]]


()

### 4. Singular Value Decomposition (SVD)

In [84]:
let array_d2 = array![
    [-1.01,  0.86, -4.60],
    [ 3.98,  0.53, -7.04],
    [ 3.98,  0.53, -7.04],
];
match array_d2.svd(true, true) {
    Ok((u, sigma, vt)) => {
        println!("The left singular vectors are: {:?}", u.unwrap());
        println!("The right singular vectors are: {:?}", vt.unwrap());
        println!("The sigma vector: {:?}", sigma);
    }
    Err(err) => {
        println!("{err}");
    }
}

The left singular vectors are: [[-0.3167331446091065, -0.948514688924756, 0.0],
 [-0.6707011685937435, 0.22396415437963857, -0.7071067811865476],
 [-0.6707011685937436, 0.2239641543796386, 0.7071067811865475]], shape=[3, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2
The right singular vectors are: [[-0.4168301381758514, -0.0816682352525302, 0.9053081990455173],
 [0.8982609360852509, -0.18954008048752713, 0.39648688325344433],
 [0.13921180485702067, 0.9784706726517249, 0.1523654033858462]], shape=[3, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2
The sigma vector: [12.040590078046721, 3.051178554664221, 9.490164740574465e-18], shape=[3], strides=[1], layout=CFcf (0xf), const ndim=1


()

### 5. Matrix Trace

In [85]:
let array_d2 = array![
    [-1.01,  0.86, -4.60],
    [ 3.98,  0.53, -7.04],
    [ 3.98,  0.53, -7.04],
];
match array_d2.trace() {
    Ok(value) => {
        println!("The sum of diagonal elements is: {:?}", value);
    }
    Err(err) => {
        println!("{err}");
    }
}

The sum of diagonal elements is: -7.52


()

### 6. Matrix Determinant

In [86]:
let array_d2 = array![
    [-1.01,  0.86, -4.60],
    [ 3.98,  0.53, -7.04],
    [ 3.98,  0.53, -7.04],
];
match array_d2.det() {
    Ok(value) => {
        println!("The detexrminant of this matrix is: {:?}", value);
    }
    Err(err) => {
        println!("{err}");
    }
}

The detexrminant of this matrix is: 2.822009292913204e-15


()

### 7. Solving Linear Equations

In [87]:
// a11x0 + a12x1 = b1    --->    3 * x0 + 2 * x1 = 1
// a21x0 + a22x1 = b2    --->    2 * x0 - 2 * x1 = -2:
let a: Array2<f64> = array![[3., 2.], [2., -2.]];
let b: Array1<f64> = array![1., -2.];
let x = a.solve_into(b).unwrap();
x

[-0.2, 0.8], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1

---
---